In [7]:
#not random, just number sequence
#1: request others name and record name (ask everyone?)
#2: generate its own name, publish it to let others confirm there is no conflicts
#3: one own channel is created, if no confilct, json format reading.
#3: if conflict, random a number to wait on their "own" chanel, and claim it as its own. the other start from 1.
#4: in their own channel, when other want to send some special message like raw data, a new channel is created.
#4: may have two different channel to send and recieve data for a client. 

In [1]:
%run util_py_2.ipynb

In [20]:
from collections import defaultdict,deque
import asyncio
class ControlA(ControlAdvance):
    def on_connect(self,client, userdata, flags, rc):
        print("ControlA:Connected with result code "+str(rc))
        client.subscribe("control")
    def subscribe(self,get:str):
        self.client.subscribe(get)
    
    def pipeline_checker(self,topic):
        #decrypted
        #should let others give a callback to it then let it use caller when state updated.
        #TODO:
        #I mean, it's good to do, but writing this kind of function is a little bit mind blowing.
        return self.pipeline.state_dict[topic]
    
    new_message=0
    def on_message(self,client,userdata,msg):
        self.pipeline(msg.topic,msg.payload)
        self.new_message+=1
        
    async def pipeline_caller(self):
        while 1:
            if self.new_message:
                self.new_message-=1
            else:
                await asyncio.sleep(0.001)
                continue
            for key,item in self.pipline.state_dict.items():
                if len(item):
                    if hasattr(self,key):
                        g=getattr(self,key)
                        g(self,self.pipeline,item.popleft())

    def update(self):
        self.pipeline_caller()
                    
    def requests_all(self,things):
        for func in self.requests_func_list:
            ret=func(self,self.pipeline,things)
            if ret is None:continue
            return ret
    
    class Channel:
        json=False
        auto_define=True
        sended=False
    
    sended_warned=0
    channels=defaultdict(Channel)
    def on_message(self,client,userdata,msg):
        topic=msg.topic
        payload=msg.payload
        
        channel=self.channels[topic]
        if channel.auto_define and not self.sended_warned and not channel.sended:
            print("impossible to auto_define without sending data first")
            self.sended_warned=1
        if channel.json:
            import json
            payload=json.loads(payload.decode())
            return self.pipeline(topic,payload)
        return self.pipeline(topic,payload)
        
        
    def publish(self,topic,payload):
        def json_transform(payload):
            import json
            return json.dumps(payload).encode()
        channel=self.channels[topic]
        
        if channel.auto_define and not channel.sended:
            channel.json=not payload is bytes
            
        
        if channel.json:
            payload=json_transform(payload)
        self.client.publish(topic,payload)
        
        channel.sended=True
        
        

In [3]:
C=ControlA()

ControlA:Connected with result code 0


In [44]:
type(b"") is bytes

True

In [7]:
def name_iterator(name_list):
    for i in range(int(10e5)):
        i=str(i)
        if i not in name_list:
            return i

In [19]:
for i in set(["1",1,"1",1,2,3]):
    print(i,end="")

1231

In [36]:
"🌻".encode()

b'\xf0\x9f\x8c\xbb'

In [41]:
import json
json.loads(json.dumps(1))

1

In [ ]:
def names(C,S,things):
    if not things=="names":return
    C.client.publish("names",C.names)
    
C.requests_func_list.append(names)

In [8]:
import asyncio
async def task_1(C,S):
    #1: request others name and record name (ask everyone?)
    C.subscribe("names")
    C.publish("requests_all","names")
    # get_others requested name
    await asyncio.sleep(1)
    li=C.pipeline_checker("names")
    eli=[]
    haved_len=0
    for i in range(len(li)):
        lii=li.pop()
        eli.extend(lii)
        if haved_len!=len(lii):
            if haved_len==0:
                haved_len=len(lii)
            else:
                print("not identical name list, should be error")
                print(lii,haved_len)
                print("="*20)
    S.names=set(eli)
    

In [51]:
s=set((1,2))
s.add(3)
s,3 in s

({1, 2, 3}, True)

In [45]:
def record_names(C,S,things):
    if not things=="name":return
    
    if  S.names:
        
        C.publish(S.name,"name already taken")
    S.names.add()
    
C.append(record_names)


async def task_2(C,S):
    #2: generate its own name, publish it to let others confirm there is no conflicts
    C.subscribe("requests_all")
    S.name=name_iterator(S.names)
    
    C.publish("name",S.name)
    C.subscribe("name")

    await asyncio.sleep(1)
    
    #3: one own channel is created, for reciving data.
    C.publish(S.name,"I should be first and only.")
    C.subscribe(S.name)
    await asyncio.sleep(1)
    
    
    
    

In [ ]:
async def task_3(C,S):
    #3:  if no confilct, json format reading.
    
    
    
    

In [ ]:
async def



In [ ]:
def name_checker(name_list,):
    

In [ ]:
class StateMachine:
    def __init__(self,):
        ...
    def state(self,):
        ...
    def next_step(self,):
        ...
    

Connected with result code 0
